# 傾向スコアマッチング

右心カテーテル（rhc : right heart catheterization）の施術有無データ

In [1]:
import numpy as np
import pandas as pd
import statsmodels.api as sm

/Users/xianshiyuming/miniconda3/lib/python3.6/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [2]:
df = pd.read_csv("http://biostat.mc.vanderbilt.edu/wiki/pub/Main/DataSets/rhc.csv")
df.head()

,Unnamed: 0,cat1,cat2,ca,sadmdte,dschdte,dthdte,lstctdte,death,cardiohx,...,meta,hema,seps,trauma,ortho,adld3p,urin1,race,income,ptid
0,1,COPD,NaN,Yes,11142,11151.0,NaN,11382,No,0,...,No,No,No,No,No,0.0,NaN,white,Under $11k,5
1,2,MOSF w/Sepsis,NaN,No,11799,11844.0,11844.0,11844,Yes,1,...,No,No,Yes,No,No,NaN,1437.0,white,Under $11k,7
2,3,MOSF w/Malignancy,MOSF w/Sepsis,Yes,12083,12143.0,NaN,12400,No,0,...,No,No,No,No,No,NaN,599.0,white,$25-$50k,9
3,4,ARF,NaN,No,11146,11183.0,11183.0,11182,Yes,0,...,No,No,No,No,No,NaN,NaN,white,$11-$25k,10
4,5,MOSF w/Sepsis,NaN,No,12035,12037.0,12037.0,12036,Yes,0,...,No,No,No,No,No,NaN,64.0,white,Under $11k,11


In [3]:
pd.crosstab(df.death,df.swang1)


swang1,No RHC,RHC
death,,
No,1315,698
Yes,2236,1486


In [4]:
df = df.iloc[:,1:]
df1 = df[['cat1','sex','death','age','swang1','meanbp1','aps1']]
df1.head()


,cat1,sex,death,age,swang1,meanbp1,aps1
0,COPD,Male,No,70.25098,No RHC,41.0,46
1,MOSF w/Sepsis,Female,Yes,78.17896,RHC,63.0,50
2,MOSF w/Malignancy,Female,No,46.09198,RHC,57.0,82
3,ARF,Female,Yes,75.33197,No RHC,55.0,48
4,MOSF w/Sepsis,Male,Yes,67.90997,RHC,65.0,72


説明変数

In [5]:
X = df1.drop(['death','swang1'],1)
dummy_X = pd.get_dummies(X[['cat1','sex']],drop_first=True)
X2 = pd.merge(X, dummy_X, left_index=True, right_index=True)
X = X2.drop(['cat1','sex'],1)

ダミー変数化

In [6]:
ps_target = df1['swang1']
dummy_ps = pd.get_dummies(ps_target)
ps_target = dummy_ps['RHC']

傾向スコア算出

In [7]:
glm = sm.Logit(ps_target, X)
result = glm.fit()
ps = pd.Series(result.predict(X))

Optimization terminated successfully.
         Current function value: inf
         Iterations 6


/Users/xianshiyuming/miniconda3/lib/python3.6/site-packages/statsmodels/discrete/discrete_model.py:1214: RuntimeWarning: overflow encountered in exp
  return 1/(1+np.exp(-X))
/Users/xianshiyuming/miniconda3/lib/python3.6/site-packages/statsmodels/discrete/discrete_model.py:1264: RuntimeWarning: divide by zero encountered in log
  return np.sum(np.log(self.cdf(q*np.dot(X,params))))


In [8]:
y = df1['death']
dummy_y = pd.get_dummies(y)
y = dummy_y['Yes']

In [9]:
ps = pd.Series(result.predict(X))
z1 = ps_target


In [10]:
table = pd.concat([ps,z1,y],axis=1)
table.columns = ['ps','treat','died']

In [11]:
interval = np.arange(0,1.05,0.05)
match_list = []
for i in range(0,len(interval)-1):
    temp0 = table[(table['treat']==0) & (interval[i] < table['ps']) & (table['ps'] < interval[i+1])]
    temp1 = table[(table['treat']==1) & (interval[i] < table['ps']) & (table['ps'] < interval[i+1])]
    if (len(temp0) > 0) & (len(temp1) > 0):
        match_list.append(temp1['died'].mean()-temp0['died'].mean())


In [12]:
np.mean(match_list)

0.040186614987502603

## IPW（傾向スコアによる重み付け推定法）

In [13]:
ps = pd.Series(result.predict(X))
z1 = ps_target
y = y
ipwe1 = sum((z1*y)/ps)/sum(z1/ps)
ipwe0 = sum(((1-z1)*y)/(1-ps))/sum((1-z1)/(1-ps))
ipwe1 - ipwe0

0.043938212689686118